#CS312 DATA ANALYTICS AND VISUALIZATION
#SUNIL BHENSPALIYA
#201951157 (CSE)
#LAB ASSIGNMENT :- 13

In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df_1=pd.read_csv('/content/sample_data/cookie_cats (2).csv');
df_1

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [ ]:
tmp=df_1.set_index("version")
data2=tmp.loc["gate_30"].reset_index()
data2["gate_40"]=tmp.loc["gate_40"].reset_index()["sum_gamerounds"]
data2["gate_30"]=tmp.loc["gate_30"].reset_index()["sum_gamerounds"]
data2=data2.iloc[:,5:]
data2

,gate_40,gate_30
0,165,3
1,1,38
2,179,0
3,187,0
4,2,39
...,...,...
44695,43,14
44696,9,7
44697,0,21
44698,5,10


# Derivation using Observed means method 

In [ ]:
observed_variance = df_1.groupby('version').mean().var()[0]
print('Observed means :', df_1.groupby('version').mean().values.ravel())
print('Variance :', observed_variance)

Observed means : [4.98756372e+06 5.24562640e+01 4.48187919e-01 1.90201342e-01
 5.00907258e+06 5.12987755e+01 4.42282750e-01 1.82000044e-01]
Variance : 231315446.31845072


In [ ]:
def perm_test(df):
    df = df.copy()
    df['sum_gamerounds'] = np.random.permutation(df['sum_gamerounds'].values)
    return df.groupby('sum_gamerounds').mean().var()[0] # calculation of variance
perm_variance = [perm_test(df_1) for _ in range(3000)]

In [ ]:
print('Pr(Prob) is', np.mean([var > observed_variance for var in perm_variance]))

Pr(Prob) is 1.0


# Using stats.f.oneway

In [ ]:
fvalue, pvalue = stats.f_oneway(data2['gate_30'],data2['gate_40'])
print("Pvalue using stats.f.oneway function",pvalue)
print("Fvalue using stats.f.oneway function",fvalue)

Pvalue using stats.f.oneway function 0.3947911230235258
Fvalue using stats.f.oneway function 0.724141283273616


# Derivation using F-stastic

In [ ]:
mu=data2.mean()
gmu=mu.mean()
print("Group wise mean of datas is ", mu.values)
print("Grand mean of data is ", gmu)

Group wise mean of datas is  [51.34217002 52.45626398]
Grand mean of data is  51.89921700223714


In [ ]:
# Degree of freedom
degf=89398 # (total number element in a group -1)*number of groups ((44700-1)*2=89398)
gdegf=1 # total number of treatments -1 (2-1=1)

In [ ]:
# Sum of Squares (SS) of error, group wise
tmp=data2-mu
tmp=tmp**2
tmp=tmp.sum().sum()
SS=1*tmp
tmp

3424724671.0055933

In [ ]:
# Mean of Sum of squares of error group wise
MSS=SS/degf 
MSS

38308.7392447884

In [ ]:
# Sum of squares of error for whole data
GSS=((mu-gmu)**2).sum()*2
tmp=(mu-gmu)**2
GSS

1.24120535111032

In [ ]:
# Mean of Sum of squares of error group wise
GMSS=GSS/gdegf 
GMSS

1.24120535111032

In [ ]:
# Fvalue
Fvalue=GMSS/MSS
Fvalue

3.2400057417164313e-05

In [ ]:
def perm_test(df):
    df = df.copy()
    df['sum_gamerounds'] = np.random.permutation(df['sum_gamerounds'].values)
    tmp=df.set_index("version")
    df2=tmp.loc["gate_30"].reset_index()
    df2["gate_40"]=tmp.loc["gate_40"].reset_index()["sum_gamerounds"]
    df2["gate_30"]=tmp.loc["gate_30"].reset_index()["sum_gamerounds"]
    df2=data2.iloc[:,5:]
    mu=df2.mean()
    SS=((df2-mu)**2).sum().sum()
    MSS=SS/degf
    GSS=((mu-gmu)**2).sum()*2
    GMSS=GSS/gdegf 
    Fvalue_perm=GMSS/MSS
    return Fvalue_perm
perm_variance = [perm_test(df_1) for _ in range(3000)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [ ]:
print('Pr(Prob) is', np.mean([fv > Fvalue for fv in perm_variance]))

Pr(Prob) is 0.0


# Derivation using Anova table

In [ ]:
model = ols("sum_gamerounds ~ version", data=df_1).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
version,3.020603e+04,1.0,0.793957,0.372909
Residual,3.431158e+09,90187.0,NaN,NaN


# Observations & Result
- As we can observe that p-value is very higher than than the alpha (i.e. 0.05) in all the derived techniques(cannot be derived through some techniques), so We should not reject the null hypothesis